In [11]:
import os

# Check these before running

# Finn's Laptop
studydir = '/mnt/d/KPUM_NODDI/Data'
derivatives = 'derivatives/dmri_op' # derivatives folder containing the processed data 
codedir = os.path.join('/home/finn', 'Code', 'kpum_noddi') # Codedir when running Notebooks

# KPUM Workstation
studydir = '/mnt/e/Finn/KPUM_NODDI/Data'
derivatives = 'derivatives/dmri_op' # derivatives folder containing the processed data 
codedir = '/home/radio/KPUM_NODDI/code/kpum_noddi' # Workaround for issue

# Finn's Laptop
studydir = '/mnt/d/KPUM_NODDI/Data'
derivatives = 'derivatives/dmri_op' # derivatives folder containing the processed data 
codedir = os.path.join('/home/finn', 'Code', 'kpum_noddi') # Codedir when running Notebooks
codedir = os.path.join(studydir, 'code', 'kpum_noddi') # Codedir when running Notebooks

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerpath = os.path.join(studydir, derivatives)
subjecttrackerfile = 'Subject_Tracker_for_dmri_dtidkinoddi_pipeline.tsv'

# Visualization script in code directory
visualizationscript = os.path.join(codedir,'shell','QC_visualize_dmri_dtidkinoddi_pipeline.sh')

###################################################################################
# USER INPUT

# Participant details
subject = '029' # e.g. '005'
session = 'MR1' #i.e. 'MR1' or 'MR2'

In [ ]:
# Create conversion file sourcedata2nifti.tsv
import os, subprocess
import pandas as pd

# Define I/O folders and files
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)
datadir=os.path.join(studydir,derivatives,os.path.join(f'sub-{subject}',f'ses-{session}'))

currdir = os.getcwd()
os.chdir(studydir)

# Read the subjecttracker file
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)

#print(df)

# Check if ready for QC
if 'Pending' in df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}')]['QC'].values :
  print(f'Starting visual QC for {datadir}')
  # Launch script for Visual QC
  p=subprocess.Popen(f"bash {visualizationscript} {subject} {session} {studydir} {derivatives}", stdout=subprocess.PIPE, shell=True)
  # Poll process.stdout to show stdout live
  while True:
    output = p.stdout.readline()
    if p.poll() is not None:
      break
    if output:
      print(output.strip().decode("utf-8"))
  rc = p.poll()
  print(f'Done visual QC for {datadir}')
  QCstatusDone = 'y'
  if QCstatusDone == 'y':
    # We update in subjectrackerfile
    df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}'), ['QC']] = 'Done'
    # sort the rows according to 'input'
    df = df.sort_values( ['session_id', 'participant_id'], ascending = [True, True])
    # and write to subjecttrackertsv-file
    df.to_csv(subjecttrackertsv, sep="\t", index=False)
elif 'Done' in df.loc[(df.participant_id == {subject}) & (df.session_id == {session})]['QC'].values :
  print(f"Visual QC already performed on {datadir}")
else:
  print(f"Status not yet ready for visual QC on {datadir}")
      
print('End of script')
os.chdir(currdir)


Starting visual QC for /mnt/d/KPUM_NODDI/Data/derivatives/dmri_op/sub-029/ses-MR1
QC of dmri_dtidkinoddi_pipeline
----------------------------
Subject:       	029
Session:        MR1
Studydir:	/mnt/d/KPUM_NODDI/Data
Derivatives: 	derivatives/dmri_op
DataDirectory:	/mnt/d/KPUM_NODDI/Data/derivatives/dmri_op/sub-029/ses-MR1

Codedir:	/mnt/d/KPUM_NODDI/Data/code/kpum_noddi/shell
/mnt/d/KPUM_NODDI/Data/code/kpum_noddi/shell/QC_visualize_dmri_dtidkinoddi_pipeline.sh   	029 MR1 /mnt/d/KPUM_NODDI/Data derivatives/dmri_op
----------------------------

############## QC of Process: Preprocess

QC of MP PCA-denosing with dwidenoise
Check the residuals! Should not contain anatomical structure in brain parenchyma

QC of Gibbs Ringing Artifacts removal with mrdegibbs
Check the residuals! Should not contain anatomical structure brain parenchyma

QC of EDDY (ORIG protocol) or TOPUP+EDDY (NEW protocol)
Check corrected dMRI, shell by shell, for residual motion, signal dropout, (excessive) image distort